In [ ]:
import pandas as pd

df = pd.read_parquet('Data/yellow_tripdata_2023-01.parquet', engine='pyarrow')



In [ ]:
before = len(df)
df = df.drop_duplicates()

after  = len(df)
removed = before - after
print(f"Duplicated file with number of {removed} was deleted")


In [ ]:
df = df[df['passenger_count'] != 0]
df = df[df['trip_distance'] != 0]

print(f"Data was clenead from rows where passenger_count = 0 and trip_distance = 0")


In [ ]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
print(f"Tpep_pickup_datetime was successfully change to datetime")


In [ ]:
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
print(f"Tpep_dropoff_datetime was successfully change to datetime")

### Create duration time to get avg in next step


In [ ]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
print(f"Creating durations times in taxi")

### Create trip_count, avg_distance and avg_duration_minutes in taxi

In [ ]:
df = df.groupby(['tpep_pickup_datetime', 'passenger_count']).agg(
    trip_count = ('tpep_pickup_datetime','count'),
    avg_distance = ('trip_distance', 'mean'),
    avg_duration_minutes = ('duration', 'mean' )  
).reset_index()
print(f"Trip_count, avg distance and avg duration minutes success")


### Saving agregate data to parquet


In [ ]:
df.to_parquet('Data/aggregated_trips.parquet')
print(f"Aggregated data was saved to parquet")

### Upload to AWS

In [ ]:
import boto3

s3 = boto3.client('s3')

bucket_name= 'mypracawsbucketsc2'
file_path ='Data/aggregated_trips.parquet'
s3_key= 'processed/aggregated_trips.parquet'

s3.upload_file(file_path, bucket_name, s3_key)
print("Upload completed.")